# Problem 1 
## Detecting Poisonous Mushrooms 

This Problem has been solved in the following journal article using SOED: 

[Self-Organizing and Error Driven (SOED) artificial neural network for smarter](classifications[https://academic.oup.com/jcde/article/4/4/282/5729001)

We will be using ucimlrepo to import the sample data. Make sure to install it:

`pip install ucimlrepo`


In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 

  
# variable information 
print(mushroom.variables) 


                        name     role         type demographic  \
0                  poisonous   Target  Categorical        None   
1                  cap-shape  Feature  Categorical        None   
2                cap-surface  Feature  Categorical        None   
3                  cap-color  Feature       Binary        None   
4                    bruises  Feature  Categorical        None   
5                       odor  Feature  Categorical        None   
6            gill-attachment  Feature  Categorical        None   
7               gill-spacing  Feature  Categorical        None   
8                  gill-size  Feature  Categorical        None   
9                 gill-color  Feature  Categorical        None   
10               stalk-shape  Feature  Categorical        None   
11                stalk-root  Feature  Categorical        None   
12  stalk-surface-above-ring  Feature  Categorical        None   
13  stalk-surface-below-ring  Feature  Categorical        None   
14    stal

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from soed import SOEDClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

### Filling missing values

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cap-shape                 8124 non-null   object
 1   cap-surface               8124 non-null   object
 2   cap-color                 8124 non-null   object
 3   bruises                   8124 non-null   object
 4   odor                      8124 non-null   object
 5   gill-attachment           8124 non-null   object
 6   gill-spacing              8124 non-null   object
 7   gill-size                 8124 non-null   object
 8   gill-color                8124 non-null   object
 9   stalk-shape               8124 non-null   object
 10  stalk-root                5644 non-null   object
 11  stalk-surface-above-ring  8124 non-null   object
 12  stalk-surface-below-ring  8124 non-null   object
 13  stalk-color-above-ring    8124 non-null   object
 14  stalk-color-below-ring  

In [4]:
# Filling Missing Values
variable_df = mushroom.variables
for c,t in variable_df.set_index('name').drop(index=['poisonous']).type.to_dict().items():
    if t == 'Categorical':
        X.loc[:,c] = np.where(X[c].isna(),X[c].mode(),X[c])
    if t == 'Continuous':
        X.loc[:,c] = X[c].fillna(X[c].median())


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cap-shape                 8124 non-null   object
 1   cap-surface               8124 non-null   object
 2   cap-color                 8124 non-null   object
 3   bruises                   8124 non-null   object
 4   odor                      8124 non-null   object
 5   gill-attachment           8124 non-null   object
 6   gill-spacing              8124 non-null   object
 7   gill-size                 8124 non-null   object
 8   gill-color                8124 non-null   object
 9   stalk-shape               8124 non-null   object
 10  stalk-root                8124 non-null   object
 11  stalk-surface-above-ring  8124 non-null   object
 12  stalk-surface-below-ring  8124 non-null   object
 13  stalk-color-above-ring    8124 non-null   object
 14  stalk-color-below-ring  

### Binary Coding Categorical Attributes

In [6]:
X = pd.get_dummies(X)
dummy_vars = [c for c in X.columns if '_' in c]
X[dummy_vars] = X[dummy_vars].astype(int)

y = np.where(y=='p',1,0)[:,0]

In [7]:
dropping_columns = []
for c in X.columns:
    if X[c].std() == 0:
        print(c)
        dropping_columns.append(c)

veil-type_p


In [8]:
X = X.drop(columns = dropping_columns)

### Standardizing Data

In [9]:
X  = (X - X.mean())/X.std()

In [10]:
X

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,2.347113,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,-0.193076,-0.404817,4.590587,-0.155572
1,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.217810,-0.155572
2,4.119631,-0.022193,-0.796161,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,5.178669,-0.404817,-0.217810,-0.155572
3,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,-0.677492,1.226430,...,2.347113,-0.994537,-0.516688,-0.795336,-0.599494,-0.337763,-0.193076,-0.404817,4.590587,-0.155572
4,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,1.667867,-0.337763,-0.193076,-0.404817,-0.217810,-0.155572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,-0.242710,-0.022193,-0.796161,2.968250,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8120,-0.242710,-0.022193,-0.796161,-0.336857,-0.062881,1.105419,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,1.005369,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8121,-0.242710,-0.022193,1.255873,-0.336857,-0.062881,-0.904523,-0.632199,-0.022193,1.475851,-0.815274,...,-0.426003,-0.994537,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572
8122,-0.242710,-0.022193,-0.796161,2.968250,-0.062881,-0.904523,-0.632199,-0.022193,-0.677492,1.226430,...,-0.426003,1.005369,-0.516688,-0.795336,-0.599494,2.960295,-0.193076,-0.404817,-0.217810,-0.155572


### PCA Transformation

In [11]:
pca = PCA(n_components=50)  # Reduce to 50 dimensions
X_pca = pd.DataFrame(pca.fit_transform(X),columns = [f'PC{i}' for i in range(1,51)])

### Split Data

In [12]:
random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.4))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

In [13]:
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50
6671,4.449360,2.663365,3.567340,-0.823407,-0.085487,0.023057,-0.943293,-0.113523,0.366588,-0.720088,...,1.196203,0.494584,-1.324174,-0.245112,0.832583,1.448653,-0.841309,0.156061,0.046698,-0.030333
5653,-1.992920,2.489138,-0.682290,-0.000367,7.123303,10.813145,8.596094,-0.016274,-6.070762,-2.019656,...,-0.160804,0.412589,-1.299452,-0.946306,0.451431,2.224776,0.613669,-0.081142,-0.620417,0.409538
5025,4.713134,-2.272777,-3.860565,0.813885,-0.317437,0.049767,0.425518,0.025135,0.195105,0.032549,...,1.430474,-0.146634,0.651371,-0.278565,0.137211,0.552796,0.595464,-0.368913,-0.085147,0.013108
4803,4.702343,-2.828507,-4.680139,-0.521890,-0.562088,0.377680,0.349845,-0.067177,-0.081055,-0.087422,...,2.222229,-1.019853,0.116663,0.031946,0.234695,0.064414,0.417315,-0.821023,-0.280639,0.558703
3851,-1.773136,-0.756552,0.130892,4.018587,-0.287603,-1.067969,1.318451,0.296851,0.154925,-0.383500,...,0.216492,1.172825,0.003266,-0.000540,-0.930226,0.706474,-0.129625,-0.358188,0.038802,-1.230754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,5.507344,-2.265148,-4.252060,-0.414996,-0.050696,0.937273,-1.164146,0.050458,-0.130313,0.097221,...,-1.076463,-1.085275,0.220704,0.324355,0.085611,-1.097516,-0.152198,-0.901455,0.042761,-0.391672
2102,-1.775696,0.196517,0.841086,3.229625,0.175919,-0.473068,0.301596,0.171327,-0.063117,-0.085154,...,-0.612671,-1.360827,-0.181324,0.922956,-0.302340,-0.415077,0.505919,-0.085669,0.110530,0.129998
182,-1.662700,-0.297485,0.602862,-0.858856,-0.766190,-0.377822,-0.972182,-0.418080,-1.432828,3.473117,...,-0.524260,0.045697,-0.198138,-0.044816,-0.504499,-0.105292,1.090697,-1.464237,0.779890,-0.296598
4411,-0.642877,-1.185154,-0.257415,-0.057723,-0.695674,0.451706,0.361610,-1.299714,-3.351850,2.048038,...,-0.837447,1.427326,-0.353116,0.776757,0.582936,-2.291604,2.165054,1.941469,0.289894,-0.942847


### Model Trainig

In [14]:
soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train)

Model training complete.


### Model Evaluation

In [15]:
y_proba = soed.predict_proba(X_test.values)
y_pred = soed.predict(X_test.values)

recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
auc = roc_auc_score(y_test,y_proba[:,1])

performance = {'recall':recall,'precision':precision,'accuracy':accuracy,'auc':auc}

In [16]:
print(performance)

{'recall': 0.9931682322801024, 'precision': 0.9995702621400946, 'accuracy': 0.9965113892879129, 'auc': np.float64(0.999913033972254)}


### Model Comparison

In [17]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','SOED'] )
report_df

,DT,SOED
repeat1,NaN,NaN
repeat2,NaN,NaN
repeat3,NaN,NaN
repeat4,NaN,NaN
repeat5,NaN,NaN


In [18]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.4))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train)
    y_proba = soed.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','SOED'] = auc

    #dt
    dt = DecisionTreeClassifier(max_depth=10)
    dt.fit(X_train.values,y_train)
    y_proba = dt.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','DT'] = auc


report_df.loc['Average'] = report_df.mean()

1
Model training complete.
2
Model training complete.
3
Model training complete.
4
Model training complete.
5
Model training complete.


In [19]:
report_df

,DT,SOED
repeat1,0.994361,0.998425
repeat2,0.994383,0.999865
repeat3,0.993999,0.997675
repeat4,0.996514,0.998321
repeat5,0.994011,0.998154
Average,0.994654,0.998488


We can see that their performance is comparable. Let's see how they will do in the next problem.

# Problem 2
## Deciding to consume mushroom 

We must assume some data to use the mushroom dataset for a decision-making problem. We will assume that when we make a false negative, we will incur a 200-dollar mistake cost; when we make a false positive, we will incur a 5-dollar cost of errors.


In [20]:
c0 = np.where(y==0,0,200)
c1 = np.where(y==1,0,5)
c = np.column_stack((c0,c1))


random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.2))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

c_train = c[train_index,:]
c_test = c[test_index,:]

In [21]:
soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train,c_train)

Model training complete.


In [22]:
y_decide = soed.decide(X_test.values)

In [23]:
df = pd.DataFrame(np.column_stack((y_test,y_decide)),columns = ['Reality','Decision'])
df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),200,0)
df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),5,0)
df['cost'] = df.FP_cost + df.FN_cost
df

,Reality,Decision,FN_cost,FP_cost,cost
0,0,0,0,0,0
1,0,0,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
6493,1,1,0,0,0
6494,0,0,0,0,0
6495,0,1,0,5,5
6496,1,1,0,0,0


In [24]:
total_cost = df.cost.sum()
print(f'total cost is {total_cost}.')

total cost is 4900.


In [25]:
df.cost.value_counts()

cost
0      5713
5       780
200       5
Name: count, dtype: int64

### Comparing Models

In [26]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','SOED'] )
report_df

,DT,SOED
repeat1,NaN,NaN
repeat2,NaN,NaN
repeat3,NaN,NaN
repeat4,NaN,NaN
repeat5,NaN,NaN


In [27]:
def calc_cost(y_reality,y_decide):
    df = pd.DataFrame(np.column_stack((y_reality,y_decide)),columns = ['Reality','Decision'])
    df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),200,0)
    df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),5,0)
    df['cost'] = df.FP_cost + df.FN_cost
    return df.cost.sum()

In [28]:
def get_cost_minimizing_threshold(y_reality,y_prob):
    candidate_df = pd.DataFrame(index=range(99),columns = ['thresh','cost'])
    candidate_df.thresh = np.linspace(0.01,0.99,99)
    candidate_df = candidate_df.set_index('thresh')
    
    for t in candidate_df.index.tolist():
        y_decide = np.where(y_prob>t,1,0)
        candidate_df.loc[t,'cost'] = calc_cost(y_reality,y_decide)
    return candidate_df[candidate_df.cost == candidate_df.cost.min()].index[0]

In [29]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.4))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    c_train = c[train_index,:]
    c_test = c[test_index,:]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train,c_train)
    y_decide = soed.decide(X_test.values)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','SOED'] = total_cost

    #dt
    dt = DecisionTreeClassifier()
    dt.fit(X_train.values,y_train)

    y_prob = dt.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob)
    print(f'dt threshod= {thresh}')
    y_prob = dt.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','DT'] = total_cost


report_df.loc['Average'] = report_df.mean()

1
Model training complete.
dt threshod= 0.01
2
Model training complete.
dt threshod= 0.01
3
Model training complete.
dt threshod= 0.01
4
Model training complete.
dt threshod= 0.01
5
Model training complete.
dt threshod= 0.01


In [30]:
report_df

,DT,SOED
repeat1,665,1020
repeat2,2020,975
repeat3,3630,2355
repeat4,2250,980
repeat5,3225,1595
Average,2358.0,1385.0


We can see that SOED will incur significantly less cost than DT.